<img style="float:right" src="https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true" alt="Insper">

# Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)
## Workshop III - Data Science Deploy

<h3 style="color:brown">Deploy: Local e em nuvem (VMs)</h3>

WEB API com Flask: Como publicar seus modelos como serviço

A documentação online do Flask e Requests são muito boas!

<img src="https://flask.palletsprojects.com/en/2.0.x/_images/flask-logo.png" width="300" style="float:left"> 
<img src="https://docs.python-requests.org/en/latest/_static/requests-sidebar.png" width="200" style="float:right">

Documentação:

- https://flask.palletsprojects.com/en/2.0.x/
- https://docs.python-requests.org/en/latest/

In [4]:
!pip install flask

In [5]:
!pip install requests

Agora vamos trabalhar fora do ambiente Jupyter, vamos trabalhar com python em arquivos texto .py, ambiente script na linha de comando. Isso funciona no Linux, Mac ou Windows.

Criar uma pasta de tralbalho (projeto), e criar o arquivo **insperds.py** que é nossa biblioteca de funções

```Python
import requests

def ddgquery(query):
    URL = "https://api.duckduckgo.com/?format=json&pretty=0&q=" + query
    response = requests.get(URL)
    ddg = response.json()
    info = ddg["Abstract"]
    if len(info) > 1:
        return info
    else:
    	return "Ops! No data found!" 

def bitcoins():
    URL="https://www.mercadobitcoin.net/api/BTC/trades/"
    response = requests.get(URL)
    return {"btc":response.json()}

def ethereum():
    URL="https://www.mercadobitcoin.net/api/ETH/trades/"
    response = requests.get(URL)
    return {"eth":response.json()}

def weather(latitude, longitude):
    KEY = "ed38d2abac6e6aded3cf1ed68fddb3c6"
    URL = f'https://api.openweathermap.org/data/2.5/weather?lat=' + str(latitude) + '&lon=' + str(longitude) + '&appid='+ KEY
    return requests.get(URL).json()
```

Agora vamos criar o script que executa as WEB APIs em Flask **core.py**

```python
# A simple Flask Hello World WEB API to get started with...

from flask import Flask
from flask import request
from datetime import datetime
import insperds

app = Flask(__name__)

@app.route('/')
def hello_world():
    now = datetime.now()
    return 'Hello from Flask, by Insper DS! - ' + str(now)

@app.route('/core')
def core():
    now = datetime.now()
    return 'Vai Corinthians!! - ' + str(now) 

@app.route('/add/<a>/<b>')
def add(a, b):
    return str(float(a) + float(b))

@app.route('/area')
def myarea():
  altura = request.args.get('altura', default = 0, type = float)
  largura = request.args.get('largura', default = 0, type = float)
  comprimento = request.args.get('comprimento', default = -1, type = float)
  
  if (comprimento < 0):
        return str(altura*largura)
  else:
        return str(altura*largura*comprimento)

@app.route('/query/<text>')
def query(text):
    return insperds.ddgquery(text)
    
@app.route('/bitcoins')
def btc():
    return insperds.bitcoins()
    
@app.route('/ethereum')
def ether():
    return insperds.ethereum()
    
@app.route('/weather/<lat>/<lon>')
def weather(lat, lon):
    return insperds.weather(lat, lon)
            
```

Precisamos instalar o Flask no sistema operacional (SO - Linux e Mac):
    
```bash
sudo apt install flask
```

Podemos executar os comandos abaixo (Linux e Mac) ou criar um script **run.sh**


```bash
export FLASK_APP=core
flask run --host=0.0.0.0
```

A janela do Flask em linha de comando fica aberta e podemos ver as requisições no log.

```bash
fito@acme-lab:/work/Insper/workshop3/flask-local$ sh run.sh
 * Serving Flask app "core"
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [1/Feb/2022 02:54:12] "GET /core HTTP/1.1" 200 -
127.0.0.1 - - [1/Feb/2022 02:54:25] "GET /add/10/333 HTTP/1.1" 200 -
127.0.0.1 - - [1/Feb/2022 02:54:38] "GET /query/brasil HTTP/1.1" 200 -


```

Podemos testar o serviço com o utilitário **curl** ou no próprio notebook com o **requests**, com o browser é possível para o verbo HTTP GET

In [8]:
!curl http://localhost:5000/core

Vai Corinthians!! - 2022-02-25 02:54:12.835312

In [9]:
!curl http://localhost:5000/add/10/333

343.0

In [10]:
!curl http://localhost:5000/query/brasil

Brazil, officially the Federative Republic of Brazil, is the largest country in both South America and Latin America. At 8.5 million square kilometers and with over 211 million people, Brazil is the world's fifth-largest country by area and the sixth most populous. Its capital is Brasília, and its most populous city is São Paulo. The federation is composed of the union of the 26 states and the Federal District. It is the largest country to have Portuguese as an official language and the only one in the Americas; it is also one of the most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world; as well as the most populous Roman Catholic-majority country. Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers. It borders all other countries in South America except Ecuador and Chile and covers 47.3% of the continent's land area.

Agora o texto usando o requests no Jupyter notebook:

In [15]:
import requests

r=requests.get("http://localhost:5000/")
r.text

'Hello from Flask, by Insper DS! - 2022-02-25 02:57:58.574631'

O processo para colocar o serviço no ar em nosso computador local (Linux ou Mec) é igual para colocar estes serviços em uma máquina virtual (servidor) em nuvem.

O ambiente em nuvem precisamos configurar o domínio (DNS) e a camada de seguranças (Firewalls etc...), conforme cada provedor.

### Máquina Virtual (VM) em nuvem

Linux Ubuntu - https://ubuntu.com/

Testes no Oracle Cloud:

```bash
root@web-app-server-01:~# hostname
web-app-server-01
root@web-app-server-01:~# uptime
 01:37:04 up 10 days,  6:37,  1 user,  load average: 0.02, 0.01, 0.00
root@web-app-server-01:~# uname -a
Linux web-app-server-01 5.11.0-1027-oracle #30~20.04.1-Ubuntu SMP Thu Jan 13 09:43:40 UTC 2022 aarch64 aarch64 aarch64 GNU/Linux
root@web-app-server-01:~# python3 --version
Python 3.8.10
```

Testes no Microsoft Azure:

```bash
fitoplancton@app-07:~$ hostname
app-07
fitoplancton@app-07:~$ uptime
 02:36:50 up 370 days, 20:49,  1 user,  load average: 0.00, 0.03, 0.02
fitoplancton@app-07:~$ uname -a
Linux app-07 4.18.0-1013-azure #13~18.04.1-Ubuntu SMP Thu Feb 28 23:48:47 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
fitoplancton@app-07:~$ python3 --version
Python 3.6.9
```

Podemos conectar so servidor Linux (VM) em nuvem pela linha de comando usando o **ssh** no Linux e Mac:


Oracle...<br>
**ssh -i ssh-key-2022-02-13.key ubuntu@w1.code.eng.br**

Microsoft...<br>
**ssh fitoplancton@lab.code.eng.br**

Para client Windows utilizamos o PuTTY - https://www.putty.org/

Note que as versões do Python disponíveis são diferentes!!! E não foi instalado com o Anaconda.

Vamos ver na máquina local:

In [16]:
!python --version

Python 3.6.13 :: Anaconda, Inc.


**LAB!** Vamos para o ambiente em núvem!!!

 > Será realizada demonstração de alguns tópicos do workshop. Os serviços em núvem requerem registro e cadastro com catão de crédito e em alguns casos CNPJ também. E vamos experimentar serviços de diferentes provedores... há riscos financeiros ao utilizar ambientes de *cloud computing* mesmo que para estudo e testes.

### Microsoft Azure - VM Linux

Conectando via SSH (Linha de comando)

```bash
fito@acme-lab:/work/Insper/Insper-Data-Science-Deploy$ ssh fitoplancton@lab.code.eng.br
fitoplancton@lab.code.eng.br's password: 
Welcome to Ubuntu 18.04.2 LTS (GNU/Linux 4.18.0-1013-azure x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Fri Feb 25 12:26:16 UTC 2022

  System load:  0.1                Processes:           118
  Usage of /:   31.0% of 28.90GB   Users logged in:     0
  Memory usage: 37%                IP address for eth0: 10.0.0.4
  Swap usage:   5%

 * Ubuntu's Kubernetes 1.14 distributions can bypass Docker and use containerd
   directly, see https://bit.ly/ubuntu-containerd or try it now with

     snap install microk8s --channel=1.14/beta --classic

  Get cloud support with Ubuntu Advantage Cloud Guest:
    http://www.ubuntu.com/business/services/cloud

 * Canonical Livepatch is available for installation.
   - Reduce system reboots and improve kernel security. Activate at:
     https://ubuntu.com/livepatch

131 packages can be updated.
3 updates are security updates.


*** System restart required ***
Last login: Thu Feb 24 02:21:13 2022 from 201.83.118.214
fitoplancton@app-07:~$ 

```

In [1]:
!curl http://lab.code.eng.br:5000/

Hello from Flask, by Insper DS! - 2022-02-25 12:32:52.003817

In [2]:
!curl http://lab.code.eng.br:5000/query/Insper

The Insper Learning Institution is a private college, that offers Higher Education in fields of Business, Economics, Engineering, Law and Computer Science in Latin America, located in São Paulo, Brazil.

Console do Flask no servidor Linux na Macrosoft Azure (Note que está sem chave SSL, protocolo HTTPS)

<img src="img/flask-azure.png">

Este serviço pode ser acessado de algumas formas diferentes, pelo IP, pelo domínio (DNS), ou pelo domínio fornecido pelo provedor:

In [3]:
!curl http://13.77.179.189:5000/add/23/44

67.0

In [6]:
!curl http://lab.code.eng.br:5000/add/66/124

190.0

In [8]:
!curl http://eng.westus2.cloudapp.azure.com:5000/add/99.99/-244

-144.01

### Rede e Segurança

Para o domínio de lab (válido na Internet) **code.eng.br**, configuramos os IPs dos serviços em núvem no gerenciador de DNS (Domínios), apontado os IPs para um subdomínio:
<img src="img/dns-subdominio.png">
Esta configuração é feita no ambiente WEB do provedor de DNS.

Um domínio custa por volta de 30,00 a 40,00 reais ao ano.

Verificando no Bash (Linux) para onde o domínio aponta:

```bash
root@acme-lab:/work/Insper$ nslookup w1.code.eng.br
Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
Name:	w1.code.eng.br
Address: 129.153.157.106

```

Podemos também verificar a propriedade do domínio:

```bash
root@acme-lab:/work/Insper$ whois w1.code.eng.br

% Copyright (c) Nic.br
%  The use of the data below is only permitted as described in
%  full by the terms of use at https://registro.br/termo/en.html ,
%  being prohibited its distribution, commercialization or
%  reproduction, in particular, to use it for advertising or
%  any similar purpose.
%  2022-02-25T02:21:57-03:00 - IP: 2804:14c:c6:878d:a64b:38e2:46b0:15de

domain:      code.eng.br
owner:       Insper
ownerid:     333.777.999-11
country:     BR
owner-c:     ATLME159
tech-c:      ATLME159
nserver:     ns1.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
nserver:     ns2.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
nserver:     ns3.locaweb.com.br
nsstat:      20220222 AA
nslastaa:    20220222
saci:        yes
created:     20200629 #21281503
changed:     20210610
expires:     20220629
status:      published
provider:    LOCAWEB (2)

nic-hdl-br:  ATLME159
person:      Insper
e-mail:      danielscarvalho@gmail.com
country:     BR
created:     20200629
changed:     20200629
provider:    LOCAWEB (2)

% Security and mail abuse issues should also be addressed to
% cert.br, http://www.cert.br/ , respectivelly to cert@cert.br
% and mail-abuse@cert.br
%
% whois.registro.br accepts only direct match queries. Types
% of queries are: domain (.br), registrant (tax ID), ticket,
% provider, CIDR block, IP and ASN.
```

Alguns provedores tem um nível de firewall (1) configurado via WEB em sua central de controle.<br>
Outros provedores tem dois nívels de seguranças (2) com o software de firewall adicional na máquina virtual (VM) o [iptables](https://www.netfilter.org/), como no Oracle Cloud por exemplo.
<img src="img/firewall-diagram.png">
É necessário nos serviços em núvem, liberar as portas para acesso aos serviços, note que neste caso as portas 80, 8080 e 5000 estão leberadas para entradas, mas todas as outras estão bloqueadas por seguranças:
<img src="img/regras-firewall.png">

Há outros aspectos de segurança que são importantes como chave de criptografia (SSL, TTL), ACL (Access Control List), etc...

Quando a chamada HTTP é feita para o serviço do Flask que está no servidor, note que o mair tempo se leva na espera do processamento do servidor para realizar computação:

<img src="img/network-timing.png">

Este nível de detalhe pode ser visto no console do browser, ferramenta utlizada por developers. Techas **Ctrl + Shift + i**.

Todos os ambientes do provedores serviços em nuvem, fornecem máquinas virtuais (VM).

São serviços mais flexíveis porem você é responsável por instalar, manter, atualizar, configura, etc... é como ter o seu próprio computador exposto na Internet 24x7x365.

O preço destes serviço variam conforme a capacidade de processamento das VMs, processador, disco, memória e banda Internet. O ambiente do Google Cloud para VMs é chamado *Google Compute Engine*:

<img src="https://cloud.google.com/languages/images/compute-single-instance.svg">

https://cloud.google.com/python/docs/getting-started/getting-started-on-compute-engine

### Extra

Dica senha segura:

Ao usar serviços na Internet, use senhas seguras geradas automaticamente e também use gerenciador de senhas.


A sua senha vai ser quebrada!!<br>
ACREDITE NISSO!!!

- https://passwordsgenerator.net/ >> Para não quebrar a sua senha!
- https://keepass.info/ >> Para você lembrar a senha
